In [2]:
!pip install polygon-api-client

     |████████████████████████████████| 53 kB 1.7 MB/s 
     |████████████████████████████████| 111 kB 49.5 MB/s 


In [20]:
import datetime
import pandas as pd
import numpy as np
from polygon import RESTClient


def ts_to_datetime(ts) -> str:
    return datetime.datetime.fromtimestamp(ts / 1000.0).strftime('%Y-%m-%d %H:%M')


def main():
    #REPLACE KEY
    key = "oMdCT9cti_5YvrUaIjZiuhFv6XcXataQ"
    folderName = 'CryptoData/'
    tickers = ['X:BTCUSD']
    dates = []
    opening = []
    closing = []
    highest = []
    lowest = []
    v = []
    vw = []
    n = []

    # RESTClient can be used as a context manager to facilitate closing the underlying http session
    # https://requests.readthedocs.io/en/master/user/advanced/#session-objects
    with RESTClient(key) as client:
        from_ = "2015-01-01"
        to = "2022-02-01"
        for tic in tickers:
          resp = client.crypto_aggregates(tic, 1, "day", from_, to, unadjusted=False)
          print(f"Hour aggregates for {resp.ticker} between {from_} and {to}.")

          for result in resp.results:
              dt = ts_to_datetime(result["t"])
              dates.append(dt)
              opening.append(result['o'])
              highest.append(result['h'])
              lowest.append(result['l'])
              closing.append(result['c'])
              v.append(result['v'])
              vw.append(result['vw'])
              n.append(result['n'])
          df = pd.DataFrame(np.transpose([dates,opening,closing,highest,lowest,n,v,vw]), columns = ['Date','O','C','H','L','N','V','VW'])
          df.to_excel(folderName + tic.split(':')[-1][:-3]+'.xlsx')
          print(tic,' completed.')

if __name__ == '__main__':
    main()

Hour aggregates for X:BTCUSD between 2015-01-01 and 2022-02-01.
X:BTCUSD  completed.


## Tweepy

In [ ]:
# Python Script to Extract tweets of a
# particular Hashtag using Tweepy and Pandas
# import modules
import pandas as pd
import tweepy


# function to display data of each tweet
def printtweetdata(n, ith_tweet):
	print()
	print(f"Tweet {n}:")
	print(f"Username:{ith_tweet[0]}")
	print(f"Tweet Text:{ith_tweet[1]}")
	print(f"Hashtags Used:{ith_tweet[-1]}")


# function to perform data extraction
def scrape(words, date_since, date_till, numtweet):
	
	# Creating DataFrame using pandas
	db = pd.DataFrame(columns=['username', 'text', 'retweeted','tweet_time', 'description(bio)', 'acc_created', 'friends', 'followers', 'retweet_count','likes_count','hashtags'])
	
	# We are using .Cursor() to search through twitter for the required tweets.
	# The number of tweets can be restricted using .items(number of tweets)
	for k in range(len(words)):
		tweets = tweepy.Cursor(api.search, q=words[k], lang="en",
							since=date_since, tweet_mode='extended').items(numtweet)
		
		# .Cursor() returns an iterable object. Each item in
		# the iterator has various attributes that you can access to
		# get information about each tweet
		list_tweets = [tweet for tweet in tweets]
		
		# Counter to maintain Tweet Count
		i = 1
		
		# we will iterate over each tweet in the list for extracting information about each tweet
		for tweet in list_tweets:
			username = tweet.user.screen_name
			hashtags = tweet.entities['hashtags']
			# print(tweet.created_at, tweet.user.description, tweet.user.created_at, tweet.user.friends_count, tweet.user.followers_count)
			
			# Retweets can be distinguished by a retweeted_status attribute,
			# in case it is an invalid reference, except block will be executed
			try:
				text = tweet.retweeted_status.full_text
				retweeted = 'True'
			except AttributeError:
				text = tweet.full_text
				retweeted = 'False'
			
			hashtext = list()
			for j in range(0, len(hashtags)):
				hashtext.append(hashtags[j]['text'])
		
		  
			
			# Here we are appending all the extracted information in the DataFrame
			ith_tweet = [username, text, retweeted, tweet.created_at, tweet.user.description, tweet.user.created_at, tweet.user.friends_count, tweet.user.followers_count, tweet.retweet_count, tweet.favorite_count, hashtext]
			
			db.loc[len(db)] = ith_tweet
			
			# Function call to print tweet data on screen
			# printtweetdata(i, ith_tweet)
			if i==int(numtweet/2):
				print('50% complete for ',words[k])
			elif i==numtweet:
				print('100% complete for ',words[k])
			i = i+1
			
	filename = 'scraped_tweets.csv'
	
	# we will save our database as a CSV file.
	db.to_csv(filename)


if __name__ == '__main__':
	
	#Pratvi credentials
  consumer_key = "pXfG7o1OfnoYaZglzT7jwmSR1"
  consumer_secret = "Ez9AbZsV2kRfKJRJpYUOmvwXlUc1MTztgEtGmyoKcDl1yDhp1g"
  access_key = "1482958101964025856-jUZJxrNboZyeq6RKVfeM4yJpSTpwhB"
  access_secret = "bspdNWX1laW2dpudQdMvK84sgfWj9NfiA8QbydGXcFVpA"

	#Vishwa credentials
	# consumer_key = "jVcRthi6uPGz07Fp255MAHfsB"
	# consumer_secret = "1jc7zeAjsWP0ecQ8to6aDkkdBr09271MihwQh69OSTmT5WdWmb"
	# access_key = "1059738121263243265-Oy5KNZBmpaUm0GMbW5fPmVy398y8oJ"
	# access_secret = "sPK9gvoyLARwW490X5blEUKImEYaNmeaIQ6yADz3muTuY"
	
  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_key, access_secret)
  api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

	# Enter Hashtag and initial date
  print("Enter Twitter HashTag to search for")
  words = ["$btc"]
  print("Enter Date since The Tweets are required in yyyy-mm--dd")
  date_since = "2017-01-01";date_till = "2022-01-01";
	
	# number of tweets you want to extract in one run
  numtweet = 15
  scrape(words, date_since,date_till, numtweet)
  print('Scraping has completed!')


Enter Twitter HashTag to search for
Enter Date since The Tweets are required in yyyy-mm--dd
50% complete for  $btc
100% complete for  $btc
Scraping has completed!


In [ ]:
data = pd.read_csv('scraped_tweets_1500.csv',encoding='ISO-8859-1', error_bad_lines=False)
data.head()

,Unnamed: 0,username,text,retweeted,tweet_time,description(bio),acc_created,friends,followers,retweet_count,likes_count,hashtags
0,0,SuarezJesse6,$BTC lol still greenð ð¼,TRUE,28-01-2022 09:24,#professional #bitcoinð #Ethereumð\nacco...,26-01-2022 08:54,235.0,23.0,1,0.0,[]
1,1,i_bot404,Just a quick update as I am so much occupied t...,FALSE,28-01-2022 09:24,Data Ninja ð¥· | private account & doesn't re...,24-10-2020 21:27,89.0,8205.0,0,0.0,"['NFA', 'DYOR']"
2,2,flodis79,"Youâre gifted $35,000 to buy the current cry...",TRUE,28-01-2022 09:24,Phunk #0321,09-04-2007 22:01,4993.0,853.0,21,0.0,[]
3,3,Is0nlyUp,@fleontrotsky And if mainstream Media were not...,FALSE,28-01-2022 09:24,watch streams at 2x. it's better.\ndon't want ...,26-02-2021 16:22,426.0,106.0,0,0.0,[]
4,4,stevenmtorok,Onchain #Bitcoin in a nutshell: Jan 27th\n\nWh...,TRUE,28-01-2022 09:24,NaN,06-02-2014 10:16,218.0,34.0,1,0.0,"['Bitcoin', 'onchain']"


## Snscrape

In [ ]:
# https://github.com/JustAnotherArchivist/snscrape.git
# !pip install snscrape
import snscrape.modules.twitter as sntwitter
import pandas as pd

In [ ]:
# Creating list to append tweet data to
tweets_list2 = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('$BTC since:2022-01-20 until:2022-01-29').get_items()):
    if i>100000:
        break
    tweets_list2.append([tweet.date, tweet.id, tweet.content, tweet.username])
    
    
# Creating a dataframe from the tweets list above
tweets_df2 = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet Id', 'Text', 'User'])
tweets_df2.tail()

,Datetime,Tweet Id,Text,User
99996,2022-01-22 17:53:31+00:00,1484947348186877957,"Holding long term stacks is one thing, but if ...",trader_bigs
99997,2022-01-22 17:53:29+00:00,1484947337969795072,@dogecoin @Bitcoin @ethereum @binance @Stellar...,KakadIshan
99998,2022-01-22 17:53:27+00:00,1484947330356944901,$BTC overall down $8000 from initial warning s...,_CryptoSmith
99999,2022-01-22 17:53:26+00:00,1484947326711971842,"Remember, $btc flashed under $4k during the pa...",ASAP17
100000,2022-01-22 17:53:21+00:00,1484947307879710724,@Bitcoin Hope so little till summer,mersinieee


In [ ]:
import itertools
# our search term, using syntax for Twitter's Advanced Search
search = '$BTC'

# the scraped tweets, this is a generator
scraped_tweets = sntwitter.TwitterSearchScraper(search).get_items()

# slicing the generator to keep only the first 100 tweets
sliced_scraped_tweets = itertools.islice(scraped_tweets, 1000)

# convert to a DataFrame and keep only relevant columns
pd.DataFrame(sliced_scraped_tweets).to_csv('btc.csv')